# LH 공고 PDF 다운로드 자동화

API로 받은 공고 목록에서 각 공고의 PDF를 자동으로 다운로드

In [2]:
# !pip install selenium webdriver-manager

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

## Chrome 드라이버 설정

In [5]:
def setup_driver(download_path):
    """
    Chrome WebDriver 설정

    Args:
        download_path: PDF 다운로드 저장 경로

    Returns:
        WebDriver 객체
    """
    chrome_options = Options()

    # 다운로드 설정
    prefs = {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True  # PDF를 브라우저에서 열지 않고 다운로드
    }
    chrome_options.add_experimental_option("prefs", prefs)

    # 헤드리스 모드 (백그라운드 실행, 필요시 주석 해제)
    # chrome_options.add_argument('--headless')

    # 드라이버 생성
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    return driver

## PDF 다운로드 함수

In [ ]:
def download_pdf_from_page(driver, url, wait_time=10):
    """
    공고 상세 페이지에서 PDF 다운로드

    Args:
        driver: WebDriver 객체
        url: 공고 상세 페이지 URL
        wait_time: 대기 시간 (초)

    Returns:
        bool: 다운로드 성공 여부
    """
    try:
        print(f"페이지 접속: {url}")
        driver.get(url)

        # 페이지 로딩 대기
        wait = WebDriverWait(driver, wait_time)

        # PDF 링크 찾기 (텍스트에 .pdf 포함된 링크)
        print("PDF 링크 찾는 중...")
        pdf_links = driver.find_elements(By.XPATH, "//a[contains(text(), '.pdf')]")

        if not pdf_links:
            print("PDF 링크를 찾을 수 없습니다.")
            return False

        # 첫 번째 PDF 링크 클릭
        pdf_link = pdf_links[0]
        print(f"PDF 링크 발견: {pdf_link.text}")

        pdf_link.click()
        print("PDF 다운로드 시작...")

        # 다운로드 완료 대기
        time.sleep(2)

        return True

    except Exception as e:
        print(f"오류 발생: {e}")
        return False

## 테스트 실행

In [6]:
# 다운로드 경로 설정
download_path = os.path.abspath("./downloads")
os.makedirs(download_path, exist_ok=True)

print(f"다운로드 경로: {download_path}")

다운로드 경로: c:\Users\diste\Documents\GitHub\SKN_19\originals\00_project\04_project\downloads


In [7]:
# 테스트 URL (API에서 받은 첫 번째 공고)
test_url = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300019062&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=10&mi=1026"

# 드라이버 설정
driver = setup_driver(download_path)

try:
    # PDF 다운로드
    success = download_pdf_from_page(driver, test_url)

    if success:
        print("다운로드 완료!")
        print(f"저장 위치: {download_path}")
    else:
        print("다운로드 실패")

finally:
    # 브라우저 종료
    time.sleep(2)
    driver.quit()
    print("브라우저 종료")

페이지 접속: https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300019062&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=10&mi=1026
PDF 링크 찾는 중...
PDF 링크 발견: [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).pdf
PDF 다운로드 시작...
다운로드 완료!
저장 위치: c:\Users\diste\Documents\GitHub\SKN_19\originals\00_project\04_project\downloads
브라우저 종료


## 다운로드된 파일 확인

In [8]:
# 다운로드 폴더의 파일 목록
files = os.listdir(download_path)
print(f"다운로드된 파일 ({len(files)}개):")
for f in files:
    file_path = os.path.join(download_path, f)
    size = os.path.getsize(file_path)
    print(f"  - {f} ({size:,} bytes)")

다운로드된 파일 (3개):
  - [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.) (1).pdf (672,570 bytes)
  - [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).md (105,708 bytes)
  - [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).pdf (672,570 bytes)
